In [1]:
import sys
import pyBigWig
print(sys.executable)

/Users/billdeng/anaconda3/envs/de_novo/bin/python


In [ ]:
from Bio import SeqIO
import os

First, read and visualize input and output data.

In [ ]:
# Get the current working directory
current_dir = os.getcwd()

# Print the current working directory
print(f"The current working directory is: {current_dir}")

In [ ]:
# Path to FASTA file
fasta_file = '../data/input.fa'

# Initialize an empty dictionary to store sequences
chromosomes = {}

# Using SeqIO.parse to read the FASTA file
for record in SeqIO.parse(fasta_file, "fasta"):
    chromosome_name = record.id
    sequence = str(record.seq)
    chromosomes[chromosome_name] = sequence

# Now, chromosomes dictionary contains all the sequences
# For example, to access the sequence for chromosome 1, you can use:
# chromosome_1_sequence = chromosomes['1']

# If you want to print out the names of all chromosomes read, you can do:
# print("Chromosomes read from the FASTA file:")
# for chromosome in chromosomes.keys():
#     print(chromosome)

In [ ]:
for chromosome in chromosomes.keys():
    print(chromosome)
    # print(type(chromosome))  # This will print the type of the chromosome variable

In [ ]:
print(type(chromosomes[chromosome_name]))

In [ ]:
chromosome_name = "chr22"  # Just an example, replace it with your actual key
# Print the type of the object and a part of the object (if it's a string, for example)
print(f"Type: {type(chromosomes[chromosome_name])}, Beginning of Content: '{chromosomes[chromosome_name][:10]}'")
print(f"Type: {len(chromosomes[chromosome_name])}, Beginning of Content: '{chromosomes[chromosome_name][1211231:1211252]}'")

Use the de novo environment.

In [ ]:
import numpy as np
import tensorflow

In [ ]:
from keras import Model
from keras import backend as K
from keras.layers import Conv2D

from keras.layers import Dense
from keras.layers import GlobalAveragePooling2D
from keras.layers import Input

In [ ]:
# from janggu import *

In [ ]:
from janggu.data import Bioseq
from janggu.data import Cover
from janggu.data import ReduceDim
from janggu.layers import DnaConv2D
from sklearn.metrics import roc_auc_score

In [ ]:
from pkg_resources import resource_filename

In [ ]:
from janggu.data import plotGenomeTrack

from janggu.data import Cover
from janggu.data import HeatTrack
from janggu.data import LineTrack

roi = resource_filename('janggu',
                        'resources/sample.bed')

bw_file = resource_filename('janggu',
                            'resources/sample.bw')

cover = Cover.create_from_bigwig('coverage1',
                                 bigwigfiles=[bw_file] * 2,
                                 conditions=['rep1', 'rep2'],
                                 roi=roi,
                                 binsize=200,
                                 stepsize=200,
                                 resolution=50)

cover2 = Cover.create_from_bigwig('coverage2',
                                  bigwigfiles=bw_file,
                                  roi=roi,
                                  binsize=200,
                                  stepsize=200,
                                  resolution=50)

plotGenomeTrack([cover, cover2],
                'chr1', 16000, 18000).savefig('coverage.png')

plotGenomeTrack([HeatTrack(cover), LineTrack(cover2)],
                'chr1', 16000, 18000).savefig('coverage2.png')

In [ ]:
import matplotlib.pyplot as plt
# Open a bigWig file
bw = pyBigWig.open("../data/aorta.bw")

# Define the region of interest
chrom = "chr1"
start = 2
end = 101000

# Retrieve values for the region
values = bw.values(chrom, start, end)

# Close the bigWig file
bw.close()

# Plot the values
plt.plot(values)
plt.xlabel("Position")
plt.ylabel("Value")
plt.title(f"bigWig Data for {chrom}:{start}-{end}")
plt.show()

### Prepare Dataset

In [ ]:
from janggu.data import Bioseq
from janggu.data import Cover
from janggu.data import ReduceDim
from janggu.data import SqueezeDim

In [ ]:
raw_label_aorta = resource_filename('janggu', '../data/aorta.bw')

We pseudo-label our train set and test set here, since it is impossible to wait for the BigWig file to split into
train set and test set. However, I have coded a split function in jupyter notebook, that when someone has more time,
could use my split function to split the BigWig data into 1_22 chromosomes and 23 chromosome. To change the whole machine
learning process, simply change the path directory to load the actually train set and test set.

For a time-efficient perspective, we will build machine learning models with pseud dataset split. Note that good results
might be due to overfitting, instead of actually capturing the feature dimensions.

In [ ]:
label_aorta = Cover.create_from_bigwig('aorta_bigwig',
                                 bigwigfiles=raw_label_aorta,
                                 roi=roi,
                                 binsize=200,
                                 resolution=None,
                                 collapser='mean')

In [ ]:
test_aorta = Cover.create_from_bigwig('aorta_bigwig',
                                 bigwigfiles=raw_label_aorta,
                                 roi=roi,
                                 binsize=200,
                                 resolution=None,
                                 collapser='mean')

In [ ]:
raw_label_artery = resource_filename('janggu', '../data/artery.bw')

In [ ]:
label_artery = Cover.create_from_bigwig('artery_bigwig',
                                 bigwigfiles=raw_label_aorta,
                                 roi=roi,
                                 binsize=200,
                                 resolution=None,
                                 collapser='mean')

In [ ]:
test_artery = Cover.create_from_bigwig('artery_bigwig',
                                 bigwigfiles=raw_label_aorta,
                                 roi=roi,
                                 binsize=200,
                                 resolution=None,
                                 collapser='mean')

In [ ]:
raw_label_pulmonic = resource_filename('janggu', '../data/pulmonic.bw')

In [ ]:
label_pulmonic = Cover.create_from_bigwig('pulmonic_bigwig',
                                 bigwigfiles=raw_label_pulmonic,
                                 roi=roi,
                                 binsize=200,
                                 resolution=None,
                                 collapser='mean')

In [ ]:
test_pulmonic = Cover.create_from_bigwig('pulmonic_bigwig',
                                 bigwigfiles=raw_label_pulmonic,
                                 roi=roi,
                                 binsize=200,
                                 resolution=None,
                                 collapser='mean')

In [ ]:
print(type(label_aorta))

In [ ]:
from janggu import Janggu

In [ ]:
fig = plotGenomeTrack([LineTrack(label_aorta)], 'chr1', 3, 290000)

In [ ]:
print(label_aorta.shape)

Prepare genome dataset

In [ ]:
genome_X = resource_filename('janggu', '../data/input.fa')

In [ ]:
# Training input and labels are purely defined genomic coordinates
DNA = Bioseq.create_from_refgenome('dna', refgenome=genome_X,
                                   roi=roi,
                                   binsize=200,
                                   cache=True)

In [ ]:
print(type(DNA))